In [10]:
import pandas as pd
import re
import numpy
import requests
pd.options.mode.chained_assignment = None  

# replace ASCII code 
get the mapping from html tutorial website

In [11]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [12]:
url = 'https://www.tutorialspoint.com/html/html_ascii_codes.htm'
r = requests.get(url, headers=header)
dfs = pd.read_html(r.text)
df = dfs[0].iloc[1:]
df.loc[df[2] == '&#32;', 0] = ''
df.loc[df[2] == '&#44;', 0] = ':'
code_map = df[[2, 0]].set_index(2).to_dict()[0]


url = 'https://www.tutorialspoint.com/html/html_color_names.htm'
r = requests.get(url, headers=header)
dfs = pd.read_html(r.text)
df = dfs[5]
df[0] = df[0].fillna(' ')
df = df.iloc[1:]
code_map.update(df[[3, 0]].set_index(3).to_dict()[0])

code_map.update({'\n': ' ', 
                 '\r': ' ', 
                 '&nbsp;': ' ', 
                 '&#160;': ' '})

In [105]:
del code_map['&#92;']

# read 10K and extract Item 1A risk factor

In [ ]:
import os
import re

## define couple patterns of regular expression

In [ ]:
def regex1(text):
    reg = 'bold;\">\s*Item 1A.\s*<.*bold;\">\s*Risk Factors.*bold;\">\s*Item 2.'
    match = re.search(reg, text, flags=re.IGNORECASE)
    if match == None:
        return match
    section = text[match.span()[0]:match.span()[1]]
    section = re.sub(r'<.*?>', '', section)
    n = section.find('Risk Factors') + 12
    section = section[n:-7]
    return section

def regex2(text):
    reg = 'ITEM 1A</B>.*RISK FACTOR.*ITEM 1B</B>'
    match = re.search(reg, text, flags=re.IGNORECASE)
    if match == None:
        return match
    section = text[match.span()[0]:match.span()[1]]
    section = re.sub(r'<.*?>', '', section)
    return section

def regex3(text):
    reg = 'Item 1A — Risk Factors</font>.*Item 1B — Unresolved Staff Comments</font>'
    match = re.search(reg, text, flags=re.IGNORECASE)
    
    if match == None:
        return match
    section = text[match.span()[0]:match.span()[1]]
    section = re.sub(r'<.*?>', '', section)
    return section

def regex4(text):
    reg = 'Item 1A.\s*Risk Factors.*ITEM 1B.'
    match = re.search(reg, text, flags=re.IGNORECASE)
    
    if match == None:
        return match
    section = text[match.span()[0]:match.span()[1]]
    section = section[:-8]
    section = re.sub(r'<.*?>', '', section)
    return section

def regex5(text):
    reg = 'Risk Factors\..*ITEM 1B.'
    match = re.search(reg, text, flags=re.IGNORECASE)
    
    if match == None:
        return match
    section = text[match.span()[0]:match.span()[1]]
    section = section[:-8]
    section = re.sub(r'<.*?>', '', section)
    return section

def regex6(text):
    reg = 'Risk Factors</font>.*>ITEM 1B'
    match = re.search(reg, text, flags=re.IGNORECASE)
    
    if match == None:
        return match
    section = text[match.span()[0]:match.span()[1]]
    section = section[:-9]
    section = re.sub(r'<.*?>', '', section)
    return section

regexs = [regex1, regex2, regex3, regex4, regex5, regex6]

In [157]:
def preprossing(text, code_map):
    text = text.strip()
    for k, v in code_map.items():
        text = text.replace(k, v)
    
    while '  ' in text:
        text = text.replace('  ', ' ')
        
    text = re.sub(r'<SMALL>', '', text, flags=re.IGNORECASE)
    text = re.sub(r'</SMALL>', '', text, flags=re.IGNORECASE)
    
    return text

In [194]:
path = './data/tenk/'
target_path = './data/riskfactor/'

In [203]:
processed = os.listdir(target_path)
processed = set(processed)

In [196]:
for name in os.listdir(path):
    if name in processed:
        continue
        
    infile = path + name
    outfile = target_path + name
    
    with open(infile) as f:
        text = f.read()
        
        text = preprossing(text, code_map)
        
        i = 0
        for reg in regexs:
            i = i + 1
            section = reg(text)
            
            if section == None:
                if i == len(regexs):
                    print(infile)
                continue
            if section != None:
                open(outfile, 'w').write(section)
                break

./data/tenk/4281-0001193125-16-470162.txt
./data/tenk/65984-0000065984-17-000067.txt
./data/tenk/1385157-0001047469-17-007037.txt
./data/tenk/66740-0001558370-18-000535.txt
./data/tenk/896878-0000896878-17-000130.txt
./data/tenk/73309-0001193125-17-062124.txt
./data/tenk/916076-0001193125-17-056282.txt
./data/tenk/35527-0001193125-17-056318.txt
./data/tenk/1123360-0001123360-18-000007.txt
./data/tenk/45012-0000045012-18-000059.txt
./data/tenk/1593538-0001193125-16-478393.txt
./data/tenk/73309-0001193125-18-064018.txt
./data/tenk/4962-0000004962-18-000032.txt
./data/tenk/1403568-0001558370-18-002733.txt
./data/tenk/1087423-0001193125-16-556503.txt
./data/tenk/103379-0001193125-16-489832.txt
./data/tenk/65984-0000065984-16-000436.txt
./data/tenk/1037868-0001193125-16-478197.txt
./data/tenk/1571949-0001571949-18-000003.txt
./data/tenk/1136869-0001193125-16-485039.txt
./data/tenk/313927-0001564590-17-002326.txt
./data/tenk/1385157-0001047469-16-016719.txt
./data/tenk/1164727-0001558370-16-